In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# finalize model and make a prediction for monthly births with random forest
import numpy as np
import pandas as pd
import plotly.express as px

from numpy import asarray
from numpy import array
from pandas import concat
from sklearn.ensemble import RandomForestRegressor
import joblib

from sklearn.metrics import mean_squared_error

In [3]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [7]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Hitachi/Data/final_data.csv"
# data_path = "final_data.csv"
data_df = pd.read_csv(data_path)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27552 entries, 0 to 27551
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      27552 non-null  int64  
 1   datetime                        27552 non-null  object 
 2   Demand (MW)                     27552 non-null  float64
 3   Temperature                     27552 non-null  float64
 4   dewpoint                        27552 non-null  float64
 5   humidity                        27552 non-null  float64
 6   precipitation                   27552 non-null  float64
 7   precipprob                      27552 non-null  float64
 8   snow                            27552 non-null  float64
 9   snowdepth                       27552 non-null  float64
 10  windgust                        27552 non-null  float64
 11  windspeed                       27552 non-null  float64
 12  sealevelpressure                

In [9]:
data_df.head()

,datetime,Demand (MW),Temperature,dewpoint,humidity,precipitation,precipprob,snow,snowdepth,windgust,...,ice,rain,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow
0,2020-01-01 00:00:00,445.8,38.0,29.0,69.76,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
1,2020-01-01 01:00:00,424.5,38.0,29.9,72.37,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
2,2020-01-01 02:00:00,423.5,38.0,30.8,75.06,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
3,2020-01-01 03:00:00,418.8,37.1,29.9,74.97,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,0,1,0,0
4,2020-01-01 04:00:00,414.8,35.1,29.0,78.41,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0


In [10]:
data_df['datetime'] = pd.to_datetime(data_df['datetime'])

# Extract date and time components
data_df['date'] = data_df['datetime'].dt.date
data_df['time'] = data_df['datetime'].dt.time

In [11]:
data_df.head()

,datetime,Demand (MW),Temperature,dewpoint,humidity,precipitation,precipprob,snow,snowdepth,windgust,...,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow,date,time
0,2020-01-01 00:00:00,445.8,38.0,29.0,69.76,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,1,0,0,0,2020-01-01,00:00:00
1,2020-01-01 01:00:00,424.5,38.0,29.9,72.37,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,1,0,0,0,2020-01-01,01:00:00
2,2020-01-01 02:00:00,423.5,38.0,30.8,75.06,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,1,0,0,0,2020-01-01,02:00:00
3,2020-01-01 03:00:00,418.8,37.1,29.9,74.97,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,1,0,0,2020-01-01,03:00:00
4,2020-01-01 04:00:00,414.8,35.1,29.0,78.41,0.0,0.0,0.0,0.0,15.410436,...,0,1,0,0,0,0,0,0,2020-01-01,04:00:00


In [17]:
data_df["time"] = data_df["time"].astype(str)

In [32]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

In [25]:
hour = "00:00:00"
hour_data_df = data_df[data_df["time"] == hour]
hour_data_df.reset_index(inplace=True)
hour_data_df.head()

,index,datetime,Demand (MW),Temperature,dewpoint,humidity,precipitation,precipprob,snow,snowdepth,...,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow,date,time
0,0,2020-01-01,445.8,38.0,29.0,69.76,0.000,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,2020-01-01,00:00:00
1,24,2020-01-02,442.9,42.2,25.1,50.45,0.000,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,2020-01-02,00:00:00
2,48,2020-01-03,482.3,51.2,48.8,91.67,0.084,100.0,0.0,0.0,...,0,0,0,0,1,0,1,0,2020-01-03,00:00:00
3,72,2020-01-04,492.3,39.8,36.0,85.95,0.000,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,2020-01-04,00:00:00
4,96,2020-01-05,485.6,29.9,26.1,85.35,0.000,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,2020-01-05,00:00:00


In [27]:
hour_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   index                           1148 non-null   int64         
 1   datetime                        1148 non-null   datetime64[ns]
 2   Demand (MW)                     1148 non-null   float64       
 3   Temperature                     1148 non-null   float64       
 4   dewpoint                        1148 non-null   float64       
 5   humidity                        1148 non-null   float64       
 6   precipitation                   1148 non-null   float64       
 7   precipprob                      1148 non-null   float64       
 8   snow                            1148 non-null   float64       
 9   snowdepth                       1148 non-null   float64       
 10  windgust                        1148 non-null   float64       
 11  wind

In [26]:
series = pd.DataFrame(hour_data_df["Demand (MW)"])
values = series.values
# Use first 10% of dataset as the test dataset
# Use 90 % of data as train dataset
# 10% of total data (27552) = 5510
first_20_percent_index = int(0.1 * len(values))
train_values = values[first_20_percent_index:, :]
test_values = values[:first_20_percent_index, :]
print(train_values.shape, test_values.shape)


# 3 weeks of data is used as prior data => 24 * 7 * 3 = 504
number_of_weeks = 3
previous_data_dependency_hours = 7*number_of_weeks
# transform the time series data into supervised learning
train_supervised_data = series_to_supervised(train_values, n_in=previous_data_dependency_hours)
test_supervised_data = series_to_supervised(test_values, n_in=previous_data_dependency_hours)
print(train_supervised_data.shape)

train_X, train_y = train_supervised_data[:, :-1], train_supervised_data[:, -1]
print(train_X.shape)

# fit model
n_estimators = 1000
model = RandomForestRegressor(n_estimators=n_estimators, verbose=1, n_jobs=-1)
model.fit(train_X, train_y)

In [ ]:
model_path = f"/content/drive/MyDrive/Colab Notebooks/Hitachi/MODEL ARCHIVE/rfc_{hour}.pkl"
save_model = True
if save_model:
  joblib.dump(model, model_path)